# Data preparation

In [1]:
from datasets import load_from_disk

def new_column(example):
    example["ner_tags"] = example["labels"]
    return example

# data = load_from_disk("test.hf")
# data = load_from_disk("testWithOrigin.hf")
data = load_from_disk("dataset.hf")
id_column = range(data.num_rows)
data = data.add_column("id", id_column)
data = data.map(new_column)

# Split up the data for testing and training
data = data.train_test_split(test_size=0.1)
test_data = data["test"]
data = data["train"].train_test_split(test_size=0.2)

print(data, test_data)

Loading cached processed dataset at /mnt/c/Users/perry/Documents/uni/Master/CompSem/project/ComputationalSemantics/dataset.hf/cache-cf6e09deaae4883c.arrow


DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'origin', 'id', 'ner_tags'],
        num_rows: 6654
    })
    test: Dataset({
        features: ['tokens', 'labels', 'origin', 'id', 'ner_tags'],
        num_rows: 1664
    })
}) Dataset({
    features: ['tokens', 'labels', 'origin', 'id', 'ner_tags'],
    num_rows: 925
})


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

example = data["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'vegetarian',
 '##s',
 'are',
 'some',
 'of',
 'the',
 'health',
 '##iest',
 'people',
 'in',
 'the',
 'world',
 '.',
 '[SEP]']

In [3]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_data = data.map(tokenize_and_align_labels, batched=True)
tokenized_data["train"][0]

Map:   0%|          | 0/6654 [00:00<?, ? examples/s]

Map:   0%|          | 0/1664 [00:00<?, ? examples/s]

{'tokens': ['Vegetarians',
  'are',
  'some',
  'of',
  'the',
  'healthiest',
  'people',
  'in',
  'the',
  'world',
  '.'],
 'labels': [-100, 1, -100, 0, 5, 5, 5, 5, -100, 5, 5, 7, 7, 0, -100],
 'origin': [1.1, None, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 7.0, 7.0, None],
 'id': 9232,
 'ner_tags': [1, 0, 5, 5, 5, 5, 5, 5, 7, 7, 0],
 'input_ids': [101,
  23566,
  2015,
  2024,
  2070,
  1997,
  1996,
  2740,
  10458,
  2111,
  1999,
  1996,
  2088,
  1012,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [4]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluation method

In [11]:
import evaluate
import numpy as np

seqeval = evaluate.load("seqeval")

#mapping = {"Theme": 1, "Agent": 2, "Patient": 3, "Experiencer": 4, "Co-Theme": 5, "Stimulus": 6, "Location": 7, "Destination": 8}
label_list = [
    "O",
    "Theme",
    "Agent",
    "Patient",
    "Experiencer",
    "Co-Theme",
    "Stimulus",
    "Location",
    "Destination",
]

labels = [label_list[i] for i in example[f"ner_tags"]]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Training

In [6]:
mapping = {"Theme": 1, "Agent": 2, "Patient": 3, "Experiencer": 4, "Co-Theme": 5, "Stimulus": 6, "Location": 7, "Destination": 8}

label2id = {"O": 0}
label2id.update(mapping)
id2label = {v: k for k, v in label2id.items()}
print(id2label)
print(label2id)
# id2label = {
#     0: "O",
#     1: "Agent",
#     2: "Location",
#     3: "Patient",
#     4: "Theme",
#     5: "Topic",
#     6: "Destination",
#     7: "Result",
# }

# label2id = {
#     "O": 0,
#     "Agent": 1,
#     "Location": 2,
#     "Patient": 3,
#     "Theme": 4,
#     "Topic": 5,
#     "Destination": 6,
#     "Result": 7,
# }

{0: 'O', 1: 'Theme', 2: 'Agent', 3: 'Patient', 4: 'Experiencer', 5: 'Co-Theme', 6: 'Stimulus', 7: 'Location', 8: 'Destination'}
{'O': 0, 'Theme': 1, 'Agent': 2, 'Patient': 3, 'Experiencer': 4, 'Co-Theme': 5, 'Stimulus': 6, 'Location': 7, 'Destination': 8}


In [7]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=9, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream

In [ ]:
# Training parameters
training_args = TrainingArguments(
    output_dir="thematic_role_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=4,
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/perrygast/.local/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.247821,0.870791,0.893353,0.881928,0.940718
2,0.064700,0.259977,0.879438,0.892978,0.886156,0.943818
3,0.034800,0.271546,0.879540,0.891100,0.885283,0.942691
4,0.031700,0.282772,0.879941,0.894480,0.887151,0.943818


/home/perrygast/.local/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Experiencer seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/perrygast/.local/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Stimulus seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/perrygast/.local/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Theme seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/perrygast/.local/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Co-Theme seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/perrygast/.local/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Location seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home

# Inference

In [41]:
from transformers import pipeline

text = "A dog chases a cat to the bus."

classifier = pipeline("ner", model="thematic_role_model/checkpoint-506")
classifier(text)

[{'entity': 'Agent',
  'score': 0.9974968,
  'index': 1,
  'word': 'a',
  'start': 0,
  'end': 1},
 {'entity': 'Agent',
  'score': 0.99777067,
  'index': 2,
  'word': 'dog',
  'start': 2,
  'end': 5},
 {'entity': 'Theme',
  'score': 0.9660569,
  'index': 4,
  'word': 'a',
  'start': 13,
  'end': 14},
 {'entity': 'Theme',
  'score': 0.9869752,
  'index': 5,
  'word': 'cat',
  'start': 15,
  'end': 18},
 {'entity': 'Destination',
  'score': 0.9916328,
  'index': 7,
  'word': 'the',
  'start': 22,
  'end': 25},
 {'entity': 'Destination',
  'score': 0.99160063,
  'index': 8,
  'word': 'bus',
  'start': 26,
  'end': 29}]

In [63]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("thematic_role_model/checkpoint-110")
inputs = tokenizer(text, return_tensors="pt")

from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("thematic_role_model/checkpoint-110")
with torch.no_grad():
    logits = model(**inputs).logits

predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

print(text)
print(inputs)
print(predicted_token_class)

A dog chases a cat to the bus.
{'input_ids': tensor([[  101,  1037,  3899, 29515,  1037,  4937,  2000,  1996,  3902,  1012,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['O', 'Agent', 'Agent', 'O', 'Theme', 'Theme', 'O', 'Destination', 'Destination', 'O', 'O']


In [37]:
import torch

In [38]:
torch.cuda.empty_cache()

In [8]:
print(torch.cuda.is_available())

print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA GeForce GTX 1050


In [28]:
tokenized_data["test"][0:10]

{'tokens': [['Susan', 'left', 'an', 'hour', 'ago', '.'],
  ['Tom', 'planted', 'three', 'apple trees', 'in', 'his', 'yard', '.'],
  ['Tom', 'arrived', 'by', 'car', '.'],
  ['I', 'was', 'relaxed', '.'],
  ['She', 'was', 'arrested', 'by', 'the', 'police', '.'],
  ['Tom', 'and', 'Mary', 'flew', 'to', 'Boston', 'yesterday', '.'],
  ['I', 'should', 'cancel', 'my', 'L.A.', 'trip', '.'],
  ['I', 'like', 'chocolate', '.'],
  ['Maria',
   'knew',
   'neither',
   'his',
   'name',
   'nor',
   'his',
   'phone number',
   '.'],
  ['I', 'pocketed', 'my', 'keys', '.']],
 'labels': [[-100, 1, 0, 0, 0, 0, 0, -100],
  [-100, 2, 0, 1, 1, -100, 0, 8, 8, 0, -100],
  [-100, 1, 0, 0, 0, 0, -100],
  [-100, 4, 0, 0, 0, -100],
  [-100, 3, 0, 0, 0, 2, 2, 0, -100],
  [-100, 1, 1, 1, 0, 0, 8, 0, 0, -100],
  [-100, 2, 0, 0, 1, 1, -100, -100, -100, 1, 0, -100],
  [-100, 4, 0, 6, 0, -100],
  [-100, 4, 0, 6, 6, 6, 6, 6, 6, -100, 0, -100],
  [-100, 2, 0, -100, 1, 1, 0, -100]],
 'origin': [[1.0, None, None, None, Non